En el primero tenemos los índices "reales" (3D); cada línea tiene el código de la persona (ejemplo "f1005_N"), el código del índice (ej "en L-en R/al L-al R x 100", que viene a ser la distancia entre endocanthions partida por la distancia entre los alares en porcentaje) y el valor. En el fichero "summary" vienen los correspondientes valores "2D" medidos en las fotografías. En cada fila tenemos el sujeto, todos los valores de los índices, y finalmente algunos datos sobre la fotografía (height y width) y la pose del sujeto en la fotografía (ángulos de Euler, la focal fx, el desplazamiento de la cámara con respecto al sujeto tx,ty,tz). 

No se trata de encontrar correspondencias entre 2D y 3D, sino ver si existe una morfologia compatible

Estimar (medir) medidas 3D a partir de medidas 2D

In [130]:
import pandas as pd
import numpy as np

image_df = pd.read_csv('summary.csv')
image_df.head()

,case,config_num,dest_dir,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,...,angle,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width
0,f1005_N,0,None,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,...,0.0,-0.0,0.0,-0.0,20,600,-100,-100,-100,800
1,f1005_N,1,None,46.010868,70.785951,69.585756,65.000000,118.038129,40.311289,28.353802,...,0.0,-0.0,0.0,-0.0,20,600,-100,-100,-50,800
2,f1005_N,5,None,40.000000,70.164642,71.383061,57.008771,105.042848,35.227830,28.149854,...,0.0,-0.0,0.0,-0.0,20,600,-100,-50,-100,800
3,f1005_N,6,None,46.000000,70.760857,72.869144,65.007692,118.067777,40.311289,28.353802,...,0.0,-0.0,0.0,-0.0,20,600,-100,-50,-50,800
4,f1005_N,7,None,53.000000,70.666667,69.586411,75.000000,137.091211,46.389654,28.252780,...,0.0,-0.0,0.0,-0.0,20,600,-100,-50,0,800


In [131]:
image_df[' config_num'].value_counts()

16      30
1803    30
3306    30
1291    30
3370    30
        ..
263      2
574      1
19       1
14       1
439      1
Name:  config_num, Length: 2090, dtype: int64

In [132]:
# Para cada case tengo que convertirlo en un vector de 25 valores (y label)
real_df = pd.read_csv('indices3d.txt', header=None, names=["case", "measurement", "value"])
real_df.head()

,case,measurement,value
0,f1005_N,al L-al R,34.995157
1,f1005_N,al L-al R/ch L-ch R x 100,69.773788
2,f1005_N,al L-al R/n-sn x 100,74.211637
3,f1005_N,ch L-ch R,50.155162
4,f1005_N,ec L-ec R,95.290407


In [133]:
real_df.case.unique()

array(['f1005_N', 'f1007_N', 'f1025_N', 'f1037_N', 'f1010_N', 'f1028_N',
       'M1002_N_conf', 'M1013_N_conf', 'M1035_N_conf', 'M1043_N_conf',
       'M1019_N_conf', 'M1041_N_conf', 'f1011_N', 'f1019_N', 'f1032_N',
       'M1004_N_conf', 'M1006_N_conf', 'f1012_N', 'f1027_N',
       'M1003_N_conf', 'M1005_N_conf', 'M1044_N_conf', 'M1007_N_conf',
       'M1009_N_conf', 'M1008_N_conf', 'M1010_N_conf', 'f1020_N',
       'f1021_N', 'f1022_N', 'f1023_N'], dtype=object)

In [134]:
image_df.case.unique()

array(['f1005_N', 'f1007_N', 'f1025_N', 'f1037_N', 'f1010_N', 'f1028_N',
       'M1002_N_conf', 'M1013_N_conf', 'M1035_N_conf', 'M1043_N_conf',
       'M1019_N_conf', 'M1041_N_conf', 'f1011_N', 'f1019_N', 'f1032_N',
       'M1004_N_conf', 'M1006_N_conf', 'f1012_N', 'f1027_N',
       'M1003_N_conf', 'M1005_N_conf', 'M1044_N_conf', 'M1007_N_conf',
       'M1009_N_conf', 'M1008_N_conf', 'M1010_N_conf', 'f1020_N',
       'f1021_N', 'f1022_N', 'f1023_N'], dtype=object)

In [135]:
real_df.case.unique()==image_df.case.unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [136]:
list(image_df)

['case',
 ' config_num',
 ' dest_dir',
 ' al L-al R',
 ' al L-al R/ch L-ch R x 100',
 ' al L-al R/n-sn x 100',
 ' ch L-ch R',
 ' ec L-ec R',
 ' en L-ec L',
 ' en L-ec L/fz L-fz R x 100',
 ' en L-en R',
 ' en L-en R/al L-al R x 100',
 ' en L-en R/ec L-ec R x 100',
 ' ft L-ft R',
 ' fz L-fz R',
 ' go L-go R',
 ' go L-go R/zy L-zy R x 100',
 ' mso L-gn',
 ' n-gn',
 ' n-gn/zy L-zy R x 100',
 ' n-sn',
 ' n-sn/n-gn x 100',
 ' sm-gn',
 ' sm-gn/mso L-gn x 100',
 ' sn-gn',
 ' zy L-zy R',
 ' zy L-zy R/ft L-ft R x 100',
 ' zy L-zy R/mso L-gn x 100',
 ' Ax',
 ' Ay',
 ' Az',
 ' angle',
 ' eulerX',
 ' eulerY',
 ' eulerZ',
 ' fx',
 ' height',
 ' tx',
 ' ty',
 ' tz',
 ' width']

En el dataset 2D (imágenes == summary) el mismo sujeto aparece un cierto número de veces, cada vez con un ligero cambio de posición (columnas ' Ax',
 ' Ay',
 ' Az',
 ' angle',
 ' eulerX',
 ' eulerY',
 ' eulerZ',
 ' fx',
 ' height',
 ' tx',
 ' ty',
 ' tz',
 ' width')
 
 todas las columnas del dataset 2d son medidas que han sido tomadas de las fotografias? Entiendo que las medidas como tal son tomadas de la foto directamente, pero el resto de columnas que son? datos sobre la fotografia
 
 width y height son la anchura y altura de la cara? no, son datos sobre la fotografia y datos del sujeto
 
 las labels y (target de predicción) aparecen en el dataset indices 3d?
 
 multi-target regression? Es decir, para cada fila hay que predecir todas y cada una de las 25 medidas que aparecen en indices 3d para el sujeto correspondiente de dicha fila?

In [137]:
image_df[image_df.case=='f1005_N']

,case,config_num,dest_dir,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,...,angle,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width
0,f1005_N,0,None,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-100,-100,800
1,f1005_N,1,None,46.010868,70.785951,69.585756,65.000000,118.038129,40.311289,28.353802,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-100,-50,800
2,f1005_N,5,None,40.000000,70.164642,71.383061,57.008771,105.042848,35.227830,28.149854,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-50,-100,800
3,f1005_N,6,None,46.000000,70.760857,72.869144,65.007692,118.067777,40.311289,28.353802,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-50,-50,800
4,f1005_N,7,None,53.000000,70.666667,69.586411,75.000000,137.091211,46.389654,28.252780,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-50,0,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,f1005_N,4056,None,70.000000,71.395127,75.199293,98.045908,181.044194,57.218878,26.599337,...,0.174533,0.124042,0.123098,-0.007654,30,600,0,50,-50,800
1980,f1005_N,4057,None,81.006173,70.429500,74.239523,115.017390,210.038092,66.272166,26.712009,...,0.174533,0.124042,0.123098,-0.007654,30,600,0,50,0,800
1981,f1005_N,4060,None,60.299254,69.548803,74.671493,86.700634,159.260792,50.990195,26.846986,...,0.174533,0.095871,0.105516,0.095871,30,600,0,50,-100,800
1982,f1005_N,4061,None,70.349129,71.976187,76.529764,97.739450,180.471604,57.070132,26.548555,...,0.174533,0.095871,0.105516,0.095871,30,600,0,50,-50,800


In [138]:
image_df.case.unique()

array(['f1005_N', 'f1007_N', 'f1025_N', 'f1037_N', 'f1010_N', 'f1028_N',
       'M1002_N_conf', 'M1013_N_conf', 'M1035_N_conf', 'M1043_N_conf',
       'M1019_N_conf', 'M1041_N_conf', 'f1011_N', 'f1019_N', 'f1032_N',
       'M1004_N_conf', 'M1006_N_conf', 'f1012_N', 'f1027_N',
       'M1003_N_conf', 'M1005_N_conf', 'M1044_N_conf', 'M1007_N_conf',
       'M1009_N_conf', 'M1008_N_conf', 'M1010_N_conf', 'f1020_N',
       'f1021_N', 'f1022_N', 'f1023_N'], dtype=object)

In [139]:
real_df.case.unique()

array(['f1005_N', 'f1007_N', 'f1025_N', 'f1037_N', 'f1010_N', 'f1028_N',
       'M1002_N_conf', 'M1013_N_conf', 'M1035_N_conf', 'M1043_N_conf',
       'M1019_N_conf', 'M1041_N_conf', 'f1011_N', 'f1019_N', 'f1032_N',
       'M1004_N_conf', 'M1006_N_conf', 'f1012_N', 'f1027_N',
       'M1003_N_conf', 'M1005_N_conf', 'M1044_N_conf', 'M1007_N_conf',
       'M1009_N_conf', 'M1008_N_conf', 'M1010_N_conf', 'f1020_N',
       'f1021_N', 'f1022_N', 'f1023_N'], dtype=object)

In [140]:
image_df.case.unique()==real_df.case.unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [141]:
np.asarray(list(image_df)[3:28])

array([' al L-al R', ' al L-al R/ch L-ch R x 100',
       ' al L-al R/n-sn x 100', ' ch L-ch R', ' ec L-ec R', ' en L-ec L',
       ' en L-ec L/fz L-fz R x 100', ' en L-en R',
       ' en L-en R/al L-al R x 100', ' en L-en R/ec L-ec R x 100',
       ' ft L-ft R', ' fz L-fz R', ' go L-go R',
       ' go L-go R/zy L-zy R x 100', ' mso L-gn', ' n-gn',
       ' n-gn/zy L-zy R x 100', ' n-sn', ' n-sn/n-gn x 100', ' sm-gn',
       ' sm-gn/mso L-gn x 100', ' sn-gn', ' zy L-zy R',
       ' zy L-zy R/ft L-ft R x 100', ' zy L-zy R/mso L-gn x 100'],
      dtype='<U26')

In [142]:
# length is 25
# todas estas medidas son comunes entre data_2d y data_3d
# se mencionan en el paper como 25 DPIs reducidos, de los 33 DPIs originales
real_df.measurement.unique()

array([' al L-al R', ' al L-al R/ch L-ch R x 100',
       ' al L-al R/n-sn x 100', ' ch L-ch R', ' ec L-ec R', ' en L-ec L',
       ' en L-ec L/fz L-fz R x 100', ' en L-en R',
       ' en L-en R/al L-al R x 100', ' en L-en R/ec L-ec R x 100',
       ' ft L-ft R', ' fz L-fz R', ' go L-go R',
       ' go L-go R/zy L-zy R x 100', ' mso L-gn', ' n-gn',
       ' n-gn/zy L-zy R x 100', ' n-sn', ' n-sn/n-gn x 100', ' sm-gn',
       ' sm-gn/mso L-gn x 100', ' sn-gn', ' zy L-zy R',
       ' zy L-zy R/ft L-ft R x 100', ' zy L-zy R/mso L-gn x 100'],
      dtype=object)

In [143]:
np.asarray(list(image_df)[3:28])==real_df.measurement.unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [144]:
GOAL INPUT: PHOTO (2d indices)
GOAL OUTPUT: can be the person filtered out? does this the person 2d photo corresponds with the 3d available indices?

    Training: model-->
INPUT: 2d photo measurement indices and other 2d photo features
OUTPUT: prediction of 3d DPIs range, and perform filtering from 2d photo

SyntaxError: invalid syntax (<ipython-input-144-5d4236d76070>, line 1)

In [145]:
# -----------------------------------------------------
# MODEL INPUT Y

real_df.head()
# create array with 25 measurement names of label y
y_measurement_names = real_df.measurement.unique()

# create list to append all the vectors that will form label y
y_values = []
y_case = []

for i in real_df.case.unique(): # 30 unique cases
#     print(i)
#     print(real_df[real_df.case==i].value.to_numpy())
#     print('')
    y_values.append(real_df[real_df.case==i].value.to_numpy())
    y_case.append(i)

# convert the list to numpy array
y_values = np.array(y_values)
y_case = np.array(y_case)
print(y_values)
print(y_case)

[[ 34.99515671  69.7737883   74.21163719  50.15516222  95.29040665
   31.27510732  27.02726827  36.51652124 104.34735741  38.32129857
  108.15297792 115.71686419 111.10568049  82.46979039 123.33906426
  105.84067154  78.56176172  47.15588825  44.55365557  23.43532085
   19.00072859  59.92123195 134.72288455 124.56696722 109.22969568]
 [ 30.80348302  63.84592363  62.76309135  48.24659316  90.33437951
   28.9344251   25.86087326  35.73876917 116.02184451  39.56275492
   94.56859261 111.88494992 109.52085543  90.00510428 121.63446435
  105.19042583  86.44632302  49.07897677  46.65726598  18.5721228
   15.26879976  57.44259045 121.68293822 128.67161799 100.03985208]
 [ 30.72208304  64.7076418   54.62489616  47.47829188  94.11979836
   32.75941876  27.34609511  33.05345152 107.58857555  35.11848952
   89.47483537 119.79560014 112.87879386  88.19164878 124.49627101
  113.93729998  89.01865443  56.24190653  49.36215492  20.29725544
   16.30350474  61.15133802 127.99261089 143.04872466 102.808

In [146]:
y_values

array([[ 34.99515671,  69.7737883 ,  74.21163719,  50.15516222,
         95.29040665,  31.27510732,  27.02726827,  36.51652124,
        104.34735741,  38.32129857, 108.15297792, 115.71686419,
        111.10568049,  82.46979039, 123.33906426, 105.84067154,
         78.56176172,  47.15588825,  44.55365557,  23.43532085,
         19.00072859,  59.92123195, 134.72288455, 124.56696722,
        109.22969568],
       [ 30.80348302,  63.84592363,  62.76309135,  48.24659316,
         90.33437951,  28.9344251 ,  25.86087326,  35.73876917,
        116.02184451,  39.56275492,  94.56859261, 111.88494992,
        109.52085543,  90.00510428, 121.63446435, 105.19042583,
         86.44632302,  49.07897677,  46.65726598,  18.5721228 ,
         15.26879976,  57.44259045, 121.68293822, 128.67161799,
        100.03985208],
       [ 30.72208304,  64.7076418 ,  54.62489616,  47.47829188,
         94.11979836,  32.75941876,  27.34609511,  33.05345152,
        107.58857555,  35.11848952,  89.47483537, 119.7956

In [147]:
y_values.shape

(30, 25)

### New processing of X_train, y_train

In [148]:
image_df

,case,config_num,dest_dir,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,...,angle,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width
0,f1005_N,0,None,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-100,-100,800
1,f1005_N,1,None,46.010868,70.785951,69.585756,65.000000,118.038129,40.311289,28.353802,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-100,-50,800
2,f1005_N,5,None,40.000000,70.164642,71.383061,57.008771,105.042848,35.227830,28.149854,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-50,-100,800
3,f1005_N,6,None,46.000000,70.760857,72.869144,65.007692,118.067777,40.311289,28.353802,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-50,-50,800
4,f1005_N,7,None,53.000000,70.666667,69.586411,75.000000,137.091211,46.389654,28.252780,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-50,0,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58291,f1023_N,4051,None,62.201286,71.195367,51.577161,87.367042,171.292148,49.648766,23.162708,...,0.174533,0.105999,0.095336,-0.105999,30,600,0,50,-50,800
58292,f1023_N,4055,None,54.009258,71.064814,50.949863,76.000000,150.029997,44.045431,23.425438,...,0.174533,0.124042,0.123098,-0.007654,30,600,0,50,-100,800
58293,f1023_N,4056,None,61.008196,70.124363,50.418275,87.000000,172.046505,50.039984,23.380873,...,0.174533,0.124042,0.123098,-0.007654,30,600,0,50,-50,800
58294,f1023_N,4060,None,53.338541,69.796671,50.522126,76.419893,150.479234,44.045431,23.333220,...,0.174533,0.095871,0.105516,0.095871,30,600,0,50,-100,800


In [149]:
def label_y(row):
    if row['case'] == y_case[0]:
        return y_values[0]
    if row['case'] == y_case[1]:
        return y_values[1]
    if row['case'] == y_case[2]:
        return y_values[2]
    if row['case'] == y_case[3]:
        return y_values[3]
    if row['case'] == y_case[4]:
        return y_values[4]
    if row['case'] == y_case[5]:
        return y_values[5]
    if row['case'] == y_case[6]:
        return y_values[6]
    if row['case'] == y_case[7]:
        return y_values[7]
    if row['case'] == y_case[8]:
        return y_values[8]
    if row['case'] == y_case[9]:
        return y_values[9]
    if row['case'] == y_case[10]:
        return y_values[10]
    if row['case'] == y_case[11]:
        return y_values[11]
    if row['case'] == y_case[12]:
        return y_values[12]
    if row['case'] == y_case[13]:
        return y_values[13]
    if row['case'] == y_case[14]:
        return y_values[14]
    if row['case'] == y_case[15]:
        return y_values[15]
    if row['case'] == y_case[16]:
        return y_values[16]
    if row['case'] == y_case[17]:
        return y_values[17]
    if row['case'] == y_case[18]:
        return y_values[18]
    if row['case'] == y_case[19]:
        return y_values[19]
    if row['case'] == y_case[20]:
        return y_values[20]
    if row['case'] == y_case[21]:
        return y_values[21]
    if row['case'] == y_case[22]:
        return y_values[22]
    if row['case'] == y_case[23]:
        return y_values[23]
    if row['case'] == y_case[24]:
        return y_values[24]
    if row['case'] == y_case[25]:
        return y_values[25]
    if row['case'] == y_case[26]:
        return y_values[26]
    if row['case'] == y_case[27]:
        return y_values[27]
    if row['case'] == y_case[28]:
        return y_values[28]
    if row['case'] == y_case[29]:
        return y_values[29]
    return 'error'
image_df.apply (lambda row: label_y(row), axis=1)

0        [34.9951567058, 69.7737882981, 74.2116371919, ...
1        [34.9951567058, 69.7737882981, 74.2116371919, ...
2        [34.9951567058, 69.7737882981, 74.2116371919, ...
3        [34.9951567058, 69.7737882981, 74.2116371919, ...
4        [34.9951567058, 69.7737882981, 74.2116371919, ...
                               ...                        
58291    [30.9650869505, 69.94197752390001, 50.44998465...
58292    [30.9650869505, 69.94197752390001, 50.44998465...
58293    [30.9650869505, 69.94197752390001, 50.44998465...
58294    [30.9650869505, 69.94197752390001, 50.44998465...
58295    [30.9650869505, 69.94197752390001, 50.44998465...
Length: 58296, dtype: object

In [150]:
image_df['y'] = image_df.apply (lambda row: label_y(row), axis=1)
image_df.head()

,case,config_num,dest_dir,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,...,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width,y
0,f1005_N,0,None,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,...,-0.0,0.0,-0.0,20,600,-100,-100,-100,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
1,f1005_N,1,None,46.010868,70.785951,69.585756,65.000000,118.038129,40.311289,28.353802,...,-0.0,0.0,-0.0,20,600,-100,-100,-50,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
2,f1005_N,5,None,40.000000,70.164642,71.383061,57.008771,105.042848,35.227830,28.149854,...,-0.0,0.0,-0.0,20,600,-100,-50,-100,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
3,f1005_N,6,None,46.000000,70.760857,72.869144,65.007692,118.067777,40.311289,28.353802,...,-0.0,0.0,-0.0,20,600,-100,-50,-50,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
4,f1005_N,7,None,53.000000,70.666667,69.586411,75.000000,137.091211,46.389654,28.252780,...,-0.0,0.0,-0.0,20,600,-100,-50,0,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."


In [151]:
image_df[3500:3501]

,case,config_num,dest_dir,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,...,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width,y
3500,f1007_N,2620,None,52.611786,67.181015,62.593172,78.313473,143.265488,50.0,28.955964,...,0.105999,-0.095336,0.105999,30,600,-100,50,-100,800,"[30.803483021999998, 63.8459236251, 62.7630913..."


In [152]:
image_df[3500:3501]

,case,config_num,dest_dir,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,...,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width,y
3500,f1007_N,2620,None,52.611786,67.181015,62.593172,78.313473,143.265488,50.0,28.955964,...,0.105999,-0.095336,0.105999,30,600,-100,50,-100,800,"[30.803483021999998, 63.8459236251, 62.7630913..."


In [153]:
image_df[6500:6501]

,case,config_num,dest_dir,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,...,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width,y
6500,f1037_N,855,None,41.10961,69.428438,66.67105,59.211485,109.55364,33.24154,25.91544,...,0.007654,0.123098,-0.124042,20,600,-50,-100,-100,800,"[34.5277991089, 67.44568711710001, 67.71112831..."


In [154]:
image_df[6500:6501]

,case,config_num,dest_dir,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,...,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width,y
6500,f1037_N,855,None,41.10961,69.428438,66.67105,59.211485,109.55364,33.24154,25.91544,...,0.007654,0.123098,-0.124042,20,600,-50,-100,-100,800,"[34.5277991089, 67.44568711710001, 67.71112831..."


In [155]:
image_df[' dest_dir'].unique()

array([' None'], dtype=object)

In [156]:
image_df[' config_num'].unique()

array([   0,    1,    5, ...,  263,  938, 1613], dtype=int64)

In [157]:
train_image_df = image_df
train_image_df = train_image_df.drop(['case', ' dest_dir'], axis=1)
train_image_df.head()

,config_num,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,en L-en R,en L-en R/al L-al R x 100,...,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width,y
0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,-0.0,0.0,-0.0,20,600,-100,-100,-100,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
1,1,46.010868,70.785951,69.585756,65.000000,118.038129,40.311289,28.353802,46.010868,100.000000,...,-0.0,0.0,-0.0,20,600,-100,-100,-50,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
2,5,40.000000,70.164642,71.383061,57.008771,105.042848,35.227830,28.149854,41.012193,102.530483,...,-0.0,0.0,-0.0,20,600,-100,-50,-100,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
3,6,46.000000,70.760857,72.869144,65.007692,118.067777,40.311289,28.353802,46.010868,100.023627,...,-0.0,0.0,-0.0,20,600,-100,-50,-50,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
4,7,53.000000,70.666667,69.586411,75.000000,137.091211,46.389654,28.252780,54.037024,101.956650,...,-0.0,0.0,-0.0,20,600,-100,-50,0,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."


In [158]:
length = len(train_image_df)

train_len = int(0.8 * length)
print(train_len)
test_len = int(0.1 * length)
print(test_len)

46636
5829


In [159]:
train_image_df.head()

,config_num,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,en L-en R,en L-en R/al L-al R x 100,...,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width,y
0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,-0.0,0.0,-0.0,20,600,-100,-100,-100,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
1,1,46.010868,70.785951,69.585756,65.000000,118.038129,40.311289,28.353802,46.010868,100.000000,...,-0.0,0.0,-0.0,20,600,-100,-100,-50,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
2,5,40.000000,70.164642,71.383061,57.008771,105.042848,35.227830,28.149854,41.012193,102.530483,...,-0.0,0.0,-0.0,20,600,-100,-50,-100,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
3,6,46.000000,70.760857,72.869144,65.007692,118.067777,40.311289,28.353802,46.010868,100.023627,...,-0.0,0.0,-0.0,20,600,-100,-50,-50,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
4,7,53.000000,70.666667,69.586411,75.000000,137.091211,46.389654,28.252780,54.037024,101.956650,...,-0.0,0.0,-0.0,20,600,-100,-50,0,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."


In [160]:
# Split the dataframe into X and y
y_label = train_image_df['y']
train_image_df_original = train_image_df
train_image_df = train_image_df.drop(['y'], axis=1)
X_label = train_image_df

In [99]:
len(y_label)
#y_label = np.random.rand(58296,1)

58296

In [100]:
X_label

,config_num,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,en L-en R,en L-en R/al L-al R x 100,...,angle,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width
0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-100,-100,800
1,1,46.010868,70.785951,69.585756,65.000000,118.038129,40.311289,28.353802,46.010868,100.000000,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-100,-50,800
2,5,40.000000,70.164642,71.383061,57.008771,105.042848,35.227830,28.149854,41.012193,102.530483,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-50,-100,800
3,6,46.000000,70.760857,72.869144,65.007692,118.067777,40.311289,28.353802,46.010868,100.023627,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-50,-50,800
4,7,53.000000,70.666667,69.586411,75.000000,137.091211,46.389654,28.252780,54.037024,101.956650,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-50,0,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58291,4051,62.201286,71.195367,51.577161,87.367042,171.292148,49.648766,23.162708,68.468971,110.076455,...,0.174533,0.105999,0.095336,-0.105999,30,600,0,50,-50,800
58292,4055,54.009258,71.064814,50.949863,76.000000,150.029997,44.045431,23.425438,60.008333,111.107492,...,0.174533,0.124042,0.123098,-0.007654,30,600,0,50,-100,800
58293,4056,61.008196,70.124363,50.418275,87.000000,172.046505,50.039984,23.380873,69.028979,113.147059,...,0.174533,0.124042,0.123098,-0.007654,30,600,0,50,-50,800
58294,4060,53.338541,69.796671,50.522126,76.419893,150.479234,44.045431,23.333220,60.207973,112.878926,...,0.174533,0.095871,0.105516,0.095871,30,600,0,50,-100,800


# Guardar X_label y y_label

In [106]:
X_label.to_csv('X_label.csv', index=False)
X_label_test = pd.read_csv('X_label.csv')
X_label_test

,config_num,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,en L-en R,en L-en R/al L-al R x 100,...,angle,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width
0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-100,-100,800
1,1,46.010868,70.785951,69.585756,65.000000,118.038129,40.311289,28.353802,46.010868,100.000000,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-100,-50,800
2,5,40.000000,70.164642,71.383061,57.008771,105.042848,35.227830,28.149854,41.012193,102.530483,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-50,-100,800
3,6,46.000000,70.760857,72.869144,65.007692,118.067777,40.311289,28.353802,46.010868,100.023627,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-50,-50,800
4,7,53.000000,70.666667,69.586411,75.000000,137.091211,46.389654,28.252780,54.037024,101.956650,...,0.000000,-0.000000,0.000000,-0.000000,20,600,-100,-50,0,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58291,4051,62.201286,71.195367,51.577161,87.367042,171.292148,49.648766,23.162708,68.468971,110.076455,...,0.174533,0.105999,0.095336,-0.105999,30,600,0,50,-50,800
58292,4055,54.009258,71.064814,50.949863,76.000000,150.029997,44.045431,23.425438,60.008333,111.107492,...,0.174533,0.124042,0.123098,-0.007654,30,600,0,50,-100,800
58293,4056,61.008196,70.124363,50.418275,87.000000,172.046505,50.039984,23.380873,69.028979,113.147059,...,0.174533,0.124042,0.123098,-0.007654,30,600,0,50,-50,800
58294,4060,53.338541,69.796671,50.522126,76.419893,150.479234,44.045431,23.333220,60.207973,112.878926,...,0.174533,0.095871,0.105516,0.095871,30,600,0,50,-100,800


In [109]:
y_label.to_pickle('y_label.csv')
y_label_test = pd.read_pickle('y_label.csv')#,squeeze = True)
y_label_test

0        [34.9951567058, 69.7737882981, 74.2116371919, ...
1        [34.9951567058, 69.7737882981, 74.2116371919, ...
2        [34.9951567058, 69.7737882981, 74.2116371919, ...
3        [34.9951567058, 69.7737882981, 74.2116371919, ...
4        [34.9951567058, 69.7737882981, 74.2116371919, ...
                               ...                        
58291    [30.9650869505, 69.94197752390001, 50.44998465...
58292    [30.9650869505, 69.94197752390001, 50.44998465...
58293    [30.9650869505, 69.94197752390001, 50.44998465...
58294    [30.9650869505, 69.94197752390001, 50.44998465...
58295    [30.9650869505, 69.94197752390001, 50.44998465...
Name: y, Length: 58296, dtype: object

In [110]:
type(y_label_test.squeeze())

pandas.core.series.Series

In [111]:
y_label_test[0]

array([ 34.99515671,  69.7737883 ,  74.21163719,  50.15516222,
        95.29040665,  31.27510732,  27.02726827,  36.51652124,
       104.34735741,  38.32129857, 108.15297792, 115.71686419,
       111.10568049,  82.46979039, 123.33906426, 105.84067154,
        78.56176172,  47.15588825,  44.55365557,  23.43532085,
        19.00072859,  59.92123195, 134.72288455, 124.56696722,
       109.22969568])

In [112]:
y_label[0]

array([ 34.99515671,  69.7737883 ,  74.21163719,  50.15516222,
        95.29040665,  31.27510732,  27.02726827,  36.51652124,
       104.34735741,  38.32129857, 108.15297792, 115.71686419,
       111.10568049,  82.46979039, 123.33906426, 105.84067154,
        78.56176172,  47.15588825,  44.55365557,  23.43532085,
        19.00072859,  59.92123195, 134.72288455, 124.56696722,
       109.22969568])

In [129]:
y_label[0][24]

109.22969568399999

In [113]:
type(y_label)

pandas.core.series.Series

In [115]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_label, y_label_test, test_size=0.2)
X_train

,config_num,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,en L-en R,en L-en R/al L-al R x 100,...,angle,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width
33343,515,33.015148,60.017622,59.938443,55.009090,102.122475,33.060551,25.992477,42.047592,127.358484,...,0.174533,0.124042,-0.123098,0.007654,20,600,-100,50,-100,800
31003,3831,68.066144,70.111696,68.582456,97.082439,174.103418,56.008928,26.798223,67.029844,98.477510,...,0.174533,0.124042,-0.123098,0.007654,30,600,0,0,-50,800
7911,3930,60.133186,68.052535,74.950141,88.362888,161.375339,52.952809,28.171968,61.032778,101.496000,...,0.174533,-0.105999,-0.095336,-0.105999,30,600,0,50,-100,800
23842,1084,75.953933,63.919434,66.084386,118.827606,186.118242,63.348244,27.817515,73.681748,97.008469,...,0.174533,0.000000,0.000000,-0.174533,20,600,-50,0,100,800
32192,1735,40.112342,67.744192,62.606791,59.211485,107.466274,34.058773,26.450673,41.436699,103.301618,...,0.174533,0.105999,-0.095336,0.105999,20,600,0,0,-100,800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48981,1074,69.000000,65.714286,63.111809,105.000000,184.000000,58.034473,24.800151,73.027392,105.836800,...,0.174533,-0.174533,0.000000,0.000000,20,600,-50,0,100,800
3231,1894,73.824115,66.616435,68.943334,110.819673,195.020512,61.351447,27.266232,81.049368,109.787117,...,0.174533,0.000000,0.000000,-0.174533,20,600,0,50,100,800
5898,3951,61.008196,65.596419,58.564345,93.005376,173.349358,63.071388,29.505811,62.072538,101.744589,...,0.174533,0.000000,-0.174533,0.000000,30,600,0,50,-50,800
416,640,38.639358,70.358121,78.790229,54.918121,100.439036,35.171011,29.055047,38.832976,100.501089,...,0.174533,0.007654,-0.123098,0.124042,20,600,-100,100,-100,800


In [116]:
import sklearn
sklearn.utils.multiclass.type_of_target(y_train[0])

'continuous'

In [117]:
# Make fake regression data to implement to the model
# (I need to check data types and sizes to feed my own data in the next box)
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# summarize dataset
print(X.shape, y.shape)
print(type(X), type(y))

(1000, 10) (1000, 2)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [118]:
X_train = np.array(X_train)
X_test = np.array(X_test)

df_train = pd.DataFrame(y_train.values.tolist(), index=y_train.index)
df_test = pd.DataFrame(y_test.values.tolist(), index=y_test.index)
y_train = np.asarray(df_train)
y_test = np.asarray(df_test)

In [119]:
print('Train')
print(X_train.shape, y_train.shape)
print(type(X_train), type(y_train))
print('Test')
print(X_test.shape, y_test.shape)
print(type(X_test), type(y_test))

Train
(46636, 39) (46636, 25)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
Test
(11660, 39) (11660, 25)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [120]:
# linear regression for multioutput regression
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
# create datasets
#X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# define model
model = LinearRegression()
# fit model
model.fit(X_train, y_train)
# make a prediction
row = X_test[100].tolist()
yhat = model.predict([row])
# summarize prediction
print(yhat[0])

[ 34.88757856  69.52879425  71.84701213  50.57704876  96.06107529
  31.04036246  26.38001144  36.91715087 106.50812005  38.40792603
 107.23952807 117.88365079 111.57133827  82.97166852 124.79407245
 106.86474635  79.60670611  48.35287529  45.29726785  22.90612469
  18.38429772  59.80429546 134.48020736 123.45979571 107.53547702]


In [121]:
y_test[100].tolist()

[34.9951567058,
 69.7737882981,
 74.2116371919,
 50.155162217,
 95.29040665059999,
 31.275107320700002,
 27.0272682731,
 36.516521242399996,
 104.347357406,
 38.3212985714,
 108.152977919,
 115.71686418600001,
 111.105680493,
 82.4697903877,
 123.33906426,
 105.840671538,
 78.5617617158,
 47.1558882542,
 44.5536555741,
 23.4353208506,
 19.0007285941,
 59.9212319511,
 134.722884551,
 124.56696721899999,
 109.22969568399999]

In [126]:
# evaluate multioutput regression model with k-fold cross-validation
from numpy import absolute
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
# create datasets
# X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# define model
# model = DecisionTreeRegressor()
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X_test, y_test, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 4.248 (0.097)


In [125]:
# evaluate multioutput regression model with k-fold cross-validation
from numpy import absolute
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
# create datasets
# X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=2, random_state=1, noise=0.5)
# define model
# model = DecisionTreeRegressor()
# define the evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X_test, y_test, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 1.422 (0.015)


## Compare linear regression for each value separately

In [161]:
train_image_df_original

,config_num,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,en L-en R,en L-en R/al L-al R x 100,...,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width,y
0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,-0.000000,0.000000,-0.000000,20,600,-100,-100,-100,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
1,1,46.010868,70.785951,69.585756,65.000000,118.038129,40.311289,28.353802,46.010868,100.000000,...,-0.000000,0.000000,-0.000000,20,600,-100,-100,-50,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
2,5,40.000000,70.164642,71.383061,57.008771,105.042848,35.227830,28.149854,41.012193,102.530483,...,-0.000000,0.000000,-0.000000,20,600,-100,-50,-100,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
3,6,46.000000,70.760857,72.869144,65.007692,118.067777,40.311289,28.353802,46.010868,100.023627,...,-0.000000,0.000000,-0.000000,20,600,-100,-50,-50,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
4,7,53.000000,70.666667,69.586411,75.000000,137.091211,46.389654,28.252780,54.037024,101.956650,...,-0.000000,0.000000,-0.000000,20,600,-100,-50,0,800,"[34.9951567058, 69.7737882981, 74.2116371919, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58291,4051,62.201286,71.195367,51.577161,87.367042,171.292148,49.648766,23.162708,68.468971,110.076455,...,0.105999,0.095336,-0.105999,30,600,0,50,-50,800,"[30.9650869505, 69.94197752390001, 50.44998465..."
58292,4055,54.009258,71.064814,50.949863,76.000000,150.029997,44.045431,23.425438,60.008333,111.107492,...,0.124042,0.123098,-0.007654,30,600,0,50,-100,800,"[30.9650869505, 69.94197752390001, 50.44998465..."
58293,4056,61.008196,70.124363,50.418275,87.000000,172.046505,50.039984,23.380873,69.028979,113.147059,...,0.124042,0.123098,-0.007654,30,600,0,50,-50,800,"[30.9650869505, 69.94197752390001, 50.44998465..."
58294,4060,53.338541,69.796671,50.522126,76.419893,150.479234,44.045431,23.333220,60.207973,112.878926,...,0.095871,0.105516,0.095871,30,600,0,50,-100,800,"[30.9650869505, 69.94197752390001, 50.44998465..."


In [228]:
s = train_image_df_original.apply(lambda x: pd.Series(x['y']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'y_single'
df = train_image_df_original.drop('y', axis=1).join(s)

In [229]:
df

,config_num,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,en L-en R,en L-en R/al L-al R x 100,...,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width,y_single
0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,-0.000000,0.000000,-0.000000,20,600,-100,-100,-100,800,34.995157
0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,-0.000000,0.000000,-0.000000,20,600,-100,-100,-100,800,69.773788
0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,-0.000000,0.000000,-0.000000,20,600,-100,-100,-100,800,74.211637
0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,-0.000000,0.000000,-0.000000,20,600,-100,-100,-100,800,50.155162
0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,-0.000000,0.000000,-0.000000,20,600,-100,-100,-100,800,95.290407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58295,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,112.671914,...,0.095871,0.105516,0.095871,30,600,0,50,-50,800,14.043200
58295,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,112.671914,...,0.095871,0.105516,0.095871,30,600,0,50,-50,800,57.820439
58295,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,112.671914,...,0.095871,0.105516,0.095871,30,600,0,50,-50,800,139.494724
58295,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,112.671914,...,0.095871,0.105516,0.095871,30,600,0,50,-50,800,162.174679


In [230]:
# there are 25 different values
df = df.reset_index()
df['y_num'] = 0
value = 0
for index, row in df.iterrows():
    #print('index', index)
    if value == 25:
        value = 0
    #print('value', value)
    df.at[index, 'y_num'] = value
    value = value + 1
df

,index,config_num,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,en L-en R,...,eulerY,eulerZ,fx,height,tx,ty,tz,width,y_single,y_num
0,0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,...,0.000000,-0.000000,20,600,-100,-100,-100,800,34.995157,0
1,0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,...,0.000000,-0.000000,20,600,-100,-100,-100,800,69.773788,1
2,0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,...,0.000000,-0.000000,20,600,-100,-100,-100,800,74.211637,2
3,0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,...,0.000000,-0.000000,20,600,-100,-100,-100,800,50.155162,3
4,0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,...,0.000000,-0.000000,20,600,-100,-100,-100,800,95.290407,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457395,58295,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,...,0.105516,0.095871,30,600,0,50,-50,800,14.043200,20
1457396,58295,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,...,0.105516,0.095871,30,600,0,50,-50,800,57.820439,21
1457397,58295,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,...,0.105516,0.095871,30,600,0,50,-50,800,139.494724,22
1457398,58295,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,...,0.105516,0.095871,30,600,0,50,-50,800,162.174679,23


In [231]:
df = df.drop(['index'], axis=1)
df

,config_num,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,en L-en R,en L-en R/al L-al R x 100,...,eulerY,eulerZ,fx,height,tx,ty,tz,width,y_single,y_num
0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,0.000000,-0.000000,20,600,-100,-100,-100,800,34.995157,0
1,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,0.000000,-0.000000,20,600,-100,-100,-100,800,69.773788,1
2,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,0.000000,-0.000000,20,600,-100,-100,-100,800,74.211637,2
3,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,0.000000,-0.000000,20,600,-100,-100,-100,800,50.155162,3
4,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,0.000000,-0.000000,20,600,-100,-100,-100,800,95.290407,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457395,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,112.671914,...,0.105516,0.095871,30,600,0,50,-50,800,14.043200,20
1457396,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,112.671914,...,0.105516,0.095871,30,600,0,50,-50,800,57.820439,21
1457397,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,112.671914,...,0.105516,0.095871,30,600,0,50,-50,800,139.494724,22
1457398,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,112.671914,...,0.105516,0.095871,30,600,0,50,-50,800,162.174679,23


In [232]:
df.to_csv('df_all_measurements.csv', index=False)
df_test = pd.read_csv('df_all_measurements.csv')
df_test

,config_num,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,en L-en R,en L-en R/al L-al R x 100,...,eulerY,eulerZ,fx,height,tx,ty,tz,width,y_single,y_num
0,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,0.000000,-0.000000,20,600,-100,-100,-100,800,34.995157,0
1,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,0.000000,-0.000000,20,600,-100,-100,-100,800,69.773788,1
2,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,0.000000,-0.000000,20,600,-100,-100,-100,800,74.211637,2
3,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,0.000000,-0.000000,20,600,-100,-100,-100,800,50.155162,3
4,0,40.000000,70.175439,70.132280,57.000000,105.042848,35.355339,28.251744,41.012193,102.530483,...,0.000000,-0.000000,20,600,-100,-100,-100,800,95.290407,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457395,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,112.671914,...,0.105516,0.095871,30,600,0,50,-50,800,14.043200,20
1457396,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,112.671914,...,0.105516,0.095871,30,600,0,50,-50,800,57.820439,21
1457397,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,112.671914,...,0.105516,0.095871,30,600,0,50,-50,800,139.494724,22
1457398,4061,61.400326,71.007952,50.913006,86.469648,171.572142,50.089919,23.423389,69.180922,112.671914,...,0.105516,0.095871,30,600,0,50,-50,800,162.174679,23


In [233]:
# Split the dataframe into X and y
all_y = df_test['y_single']
all_X = df_test.drop(['y_single'], axis=1)

all_X_train, all_X_test, all_y_train, all_y_test = train_test_split(all_X, all_y, test_size=0.2)
all_X_train

,config_num,al L-al R,al L-al R/ch L-ch R x 100,al L-al R/n-sn x 100,ch L-ch R,ec L-ec R,en L-ec L,en L-ec L/fz L-fz R x 100,en L-en R,en L-en R/al L-al R x 100,...,eulerX,eulerY,eulerZ,fx,height,tx,ty,tz,width,y_num
827627,181,41.048752,59.391232,59.843934,69.115845,124.402572,38.000000,24.783721,51.156622,124.624064,...,0.007654,0.123098,-0.124042,20,600,-100,-100,-50,800,2
1411958,66,42.000000,71.186441,54.527064,59.000000,114.004386,34.058773,24.154550,46.000000,109.523810,...,-0.000000,0.000000,-0.000000,20,600,0,50,-50,800,8
735138,373,60.406953,70.754319,66.970179,85.375641,148.973152,51.478151,29.046639,58.420887,96.712189,...,0.007654,-0.123098,0.124042,20,600,-100,0,50,800,13
1072138,442,60.074953,68.262585,72.123417,88.005682,135.014814,38.013156,20.659324,59.008474,98.224752,...,-0.124042,0.123098,0.007654,20,600,-100,0,0,800,13
458834,1232,57.218878,66.088295,63.509920,86.579443,136.619911,45.705580,27.554488,50.358713,88.010662,...,0.123102,-0.007596,-0.123102,20,600,-50,50,0,800,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514084,1836,47.095647,72.446269,67.272632,65.007692,122.409150,43.011626,30.143729,44.181444,93.812160,...,-0.124042,-0.123098,-0.007654,20,600,0,50,-50,800,9
235142,2350,54.147945,66.722317,60.744604,81.154174,143.125819,50.159745,28.322571,56.080300,103.568656,...,0.105999,-0.095336,0.105999,30,600,-100,-50,-100,800,17
1408738,3921,67.186308,73.719913,62.766277,91.137259,166.003012,50.039984,24.057407,65.030762,96.791688,...,0.124042,0.123098,-0.007654,30,600,0,0,-50,800,13
145977,3872,72.560320,66.100640,54.570516,109.772492,207.067622,70.710678,27.953470,73.681748,101.545512,...,0.123102,-0.007596,-0.123102,30,600,0,0,0,800,2


In [ ]:
# Now I have to predict linear regression with only one thing